# Backtrader: Quick Start

https://www.backtrader.com/docu/quickstart/quickstart/

## Basic Setup

In [1]:
from __future__ import (absolute_import, division, print_function, unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd

In [2]:
if __name__ == '__main__':
    cerebro = bt.Cerebro() # The Cerebro engine was instantiated
    print('Starting portfolio value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.run() # The resulting cerebro instance was told to run (loop over data)
    print('Final portfolio value: %.2f' % cerebro.broker.getvalue())

Starting portfolio value: 10000.00
Final portfolio value: 10000.00


Although it doesn’t seem much, let’s point out something explicitly shown:

The Cerebro engine has created a broker instance in the background

The instance already has some cash to start with

This behind the scenes broker instantiation is a constant trait in the platform to simplify the life of the user. If no broker is set by the user, a default one is put in place.

And 10K monetary units is a usual value with some brokers to begin with.

## Setting the cash

In [3]:
if __name__ == '__main__':
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(1000000.0) # Set custom start cash. 
    print('Starting portfolio value: %.2f' % cerebro.broker.getvalue())
    
    cerebro.run() 
    print('Final portfolio value: %.2f' % cerebro.broker.getvalue())

Starting portfolio value: 1000000.00
Final portfolio value: 1000000.00


## Adding a Data Feed

Having cash is fun, but the purpose behind all this is to let an automated strategy multiply the cash without moving a finger by operating on an asset which we see as a Data Feed

In [4]:
df = pd.read_csv("AAPL.csv")
df.head()

,date,open,high,low,close,volume
0,2014-12-31,112.820000,113.129997,110.209999,101.419060,41403400
1,2015-01-02,111.389999,111.440002,107.349998,100.454300,53204600
2,2015-01-05,108.290001,108.650002,105.410004,97.624336,64285500
3,2015-01-06,106.540001,107.430000,104.629997,97.633545,65797100
4,2015-01-07,107.199997,108.199997,106.699997,99.002556,40105900


In [5]:
'2011/23/11'.replace('/', '-')

'2011-23-11'

In [6]:
df.date = df.date.apply(lambda x: x.replace('/', '-'))
df.date = pd.to_datetime(df.date)
df.head() # yyyy-mm-dd 형태로 ohlc+adjclose+v 인 yahoo finance data format에 맞춰줌. 

,date,open,high,low,close,volume
0,2014-12-31,112.820000,113.129997,110.209999,101.419060,41403400
1,2015-01-02,111.389999,111.440002,107.349998,100.454300,53204600
2,2015-01-05,108.290001,108.650002,105.410004,97.624336,64285500
3,2015-01-06,106.540001,107.430000,104.629997,97.633545,65797100
4,2015-01-07,107.199997,108.199997,106.699997,99.002556,40105900


In [12]:
df.tail()

,date,open,high,low,close,volume
1261,2020-01-06,293.790008,299.959992,292.750000,299.799988,29596800
1262,2020-01-07,299.839996,300.899994,297.480011,298.390015,27218000
1263,2020-01-08,297.160004,304.440002,297.160004,303.190002,33019800
1264,2020-01-09,307.239990,310.429993,306.200012,309.630005,42527100
1265,2020-01-10,310.600006,312.670013,308.250000,310.329987,35161200


In [7]:
df.to_csv('AAPL.csv', index=0)

In [8]:
os.getcwd()

'E:\\VSCodeProjects\\backtrader_tutorial'

In [9]:
datapath = os.path.join(os.getcwd(), './AAPL.csv')
datapath

'E:\\VSCodeProjects\\backtrader_tutorial\\./AAPL.csv'

In [10]:
os.path.isfile(datapath)

True

Yahoo Online sends the CSV data in date descending order, which is not the standard convention. The reversed=True prameter takes into account that the CSV data in the file has already been reversed and has the standard expected date ascending order.

In [13]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    
#     modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
#     datapath = os.path.join(modpath, '../../datas/orcl-1995-2014.txt')
    datapath = os.path.join(os.getcwd(), './AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2015, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2020, 1, 10),
        reverse=False) # Yahoo data는 보통의 데이터와 반대이다. 

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
Final Portfolio Value: 100000.00


## Our First Strategy

The cash is in the broker and the Data Feed is there. It seems like risky business is just around the corner.

Let’s put a Strategy into the equation and print the “Close” price of each day (bar).

DataSeries (the underlying class in Data Feeds) objects have aliases to access the well known OHLC (Open High Low Close) daily values. This should ease up the creation of our printing logic.

In [14]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

<h3 style='color:red'> 이해하기 어려운 부분. internals를 살펴봐야 한다. </h3>

Let’s explain some of the magic:

- Upon init being called the strategy already has a list of datas that are present in the platform. This is a standard Python list and datas can be accessed in the order they were inserted. The first data in the list self.datas[0] is the default data for trading operations and to keep all strategy elements synchronized (it’s the system clock)

- self.dataclose = self.datas[0].close keeps a reference to the close line. Only one level of indirection is later needed to access the close values.

- The strategy next method will be called on each bar of the system clock (self.datas[0]). This is true until other things come into play like indicators, which need some bars to start producing an output. More on that later.

In [15]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    datapath = os.path.join(os.getcwd(), './AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2015, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2020, 1, 10),
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2015-01-02, Close, 53204600.00
2015-01-05, Close, 64285500.00
2015-01-06, Close, 65797100.00
2015-01-07, Close, 40105900.00
2015-01-08, Close, 59364500.00
2015-01-09, Close, 53699500.00
2015-01-12, Close, 49650800.00
2015-01-13, Close, 67091900.00
2015-01-14, Close, 48956600.00
2015-01-15, Close, 60014000.00
2015-01-16, Close, 78513300.00
2015-01-20, Close, 49899900.00
2015-01-21, Close, 48575900.00
2015-01-22, Close, 53796400.00
2015-01-23, Close, 46464800.00
2015-01-26, Close, 55615000.00
2015-01-27, Close, 95568700.00
2015-01-28, Close, 146477100.00
2015-01-29, Close, 84436400.00
2015-01-30, Close, 83745500.00
2015-02-02, Close, 62739100.00
2015-02-03, Close, 51915700.00
2015-02-04, Close, 70149700.00
2015-02-05, Close, 42246200.00
2015-02-06, Close, 43706600.00
2015-02-09, Close, 38889800.00
2015-02-10, Close, 62008500.00
2015-02-11, Close, 73561800.00
2015-02-12, Close, 74474500.00
2015-02-13, Close, 54272200.00
2015-02-17, Close, 63152400.00
20

## Adding some Logic to the Strategy

Let’s try some crazy idea we had by looking at some charts

If the price has been falling 3 sessions in a row … BUY BUY BUY!!!

In [16]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        if self.dataclose[0] < self.dataclose[-1]:
            # current close less than previous close

            if self.dataclose[-1] < self.dataclose[-2]:
                # previous close less than the previous close

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.buy()

In [17]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    datapath = os.path.join(os.getcwd(), './AAPL.csv')

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2015, 1, 1),
        # Do not pass values after this date
        todate=datetime.datetime(2020, 1, 10),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2015-01-02, Close, 53204600.00
2015-01-05, Close, 64285500.00
2015-01-06, Close, 65797100.00
2015-01-07, Close, 40105900.00
2015-01-08, Close, 59364500.00
2015-01-09, Close, 53699500.00
2015-01-12, Close, 49650800.00
2015-01-12, BUY CREATE, 49650800.00
2015-01-13, Close, 67091900.00
2015-01-14, Close, 48956600.00
2015-01-15, Close, 60014000.00
2015-01-16, Close, 78513300.00
2015-01-20, Close, 49899900.00
2015-01-21, Close, 48575900.00
2015-01-21, BUY CREATE, 48575900.00
2015-01-22, Close, 53796400.00
2015-01-23, Close, 46464800.00
2015-01-26, Close, 55615000.00
2015-01-27, Close, 95568700.00
2015-01-28, Close, 146477100.00
2015-01-29, Close, 84436400.00
2015-01-30, Close, 83745500.00
2015-01-30, BUY CREATE, 83745500.00
2015-02-02, Close, 62739100.00
2015-02-02, BUY CREATE, 62739100.00
2015-02-03, Close, 51915700.00
2015-02-03, BUY CREATE, 51915700.00
2015-02-04, Close, 70149700.00
2015-02-05, Close, 42246200.00
2015-02-06, Close, 43706600.00
2015-02-